## C S 329E HW 5

# Crossvalidation and hyperparameter selection

## Your name here (and your partner's name if you are working in a pair)
## Parth Dave and Kevin Ly
## Group 22

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [1]:
# Google colab's default version of scikit-learn isn't the latest, so you will 
# need to update the virtual machine and restart the runtime
!pip install scikit-learn==1.0

     |████████████████████████████████| 23.1 MB 1.9 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target value (the dependent variable) series `s_y`

In [3]:
x,y = load_diabetes(return_X_y=True)
df_X = pd.DataFrame(x)
s_y = pd.DataFrame(y)

In [4]:
df_X.head(5)

,0,1,2,3,4,5,6,7,8,9
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


In [5]:
s_y.head(5)

,0
0,151.0
1,75.0
2,141.0
3,206.0
4,135.0


## Q1.2 Define a function that creates a linear least squares regression model and a function to predict a continuous value given a linear regression model
The first function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW3).  You can not use any libraries outside of pandas and numpy. Note that the length of beta_hat should be the number of columns in `df_X` + 1. 

The second function should take in two parameters, `beta_hat` - the model generated from the `get_linear_regression` function, and `df_X` - that has the attribute values at which we want to predict a continuous value.  We assume that the format and ordering of `df_X` used to create the model and `df_X` used to generate predictions are consistent. 

In [6]:
def get_linear_regression_model( df_X, s_y ):
    return np.linalg.lstsq(pd.concat([pd.DataFrame(np.ones(len(df_X))), df_X], axis=1), s_y, rcond=None)[0]

In [7]:
pd.concat([pd.DataFrame(np.ones(len(df_X))), df_X], axis=1)

,0,0,1,2,3,4,5,6,7,8,9
0,1.0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,1.0,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,1.0,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,1.0,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,1.0,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641
...,...,...,...,...,...,...,...,...,...,...,...
437,1.0,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207
438,1.0,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018118,0.044485
439,1.0,0.041708,0.050680,-0.015906,0.017282,-0.037344,-0.013840,-0.024993,-0.011080,-0.046879,0.015491
440,1.0,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044528,-0.025930


In [8]:
# code to check beta_hat
np.random.seed(23)
beta_hat = get_linear_regression_model( pd.DataFrame(np.random.random((34,4))), pd.Series(np.random.random(34)*10.0) )
beta_hat

array([ 4.18818425,  1.77890808,  0.74032569, -1.3506416 ,  0.14535984])

In [9]:
def predict_linear_regression_value( beta_hat, df_X ):
    return pd.concat([pd.DataFrame(np.ones(len(df_X))), df_X], axis=1).dot(beta_hat)

In [10]:
predicted_vals = predict_linear_regression_value( beta_hat, pd.DataFrame(np.random.random((3,4))))
predicted_vals

0    4.036695
1    4.395026
2    3.908847
dtype: float64

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Here is some example output from checking the length of the folds:
```
Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 96 and length of series is 96
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 79 and length of series is 79
Fold 5 length of dataframe is 91 and length of series is 91
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df
```

In [11]:
def partition_data( df_X, s_y, k ):
    rand_idx = np.random.randint(1, k + 1, size=len(df_X))
    dict_k_df_X, dict_k_s_y = {}, {}
    for i in range(1, k + 1):
        dict_k_df_X[i] = df_X[rand_idx == i]
        dict_k_s_y[i] = s_y[rand_idx == i]
    return dict_k_df_X, dict_k_s_y


In [12]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [13]:
# Check fold sizes
for k in dict_k_df_X.keys():
  print("Fold {} length of dataframe is {} and length of series is {}".format(k, len(dict_k_df_X[k]), len(dict_k_s_y[k])))

Fold 1 length of dataframe is 91 and length of series is 91
Fold 2 length of dataframe is 98 and length of series is 98
Fold 3 length of dataframe is 77 and length of series is 77
Fold 4 length of dataframe is 75 and length of series is 75
Fold 5 length of dataframe is 101 and length of series is 101


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [14]:
def get_mae( s_y, s_y_hat):
    return np.abs(s_y - s_y_hat).sum() / len(s_y)

In [15]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

You must use your helper functions that you wrote above, `get_linear_regression_model`, `predict_linear_regression_value` and `get_mae`. 

In [16]:
mae = np.array([])
for k in dict_k_df_X.keys():
    test_set = dict_k_df_X[k]
    s_y = dict_k_s_y[k]

    train_set = pd.concat([val for key, val in dict_k_df_X.items() if key != k]).reset_index(drop=True)
    s_y_train = pd.concat([val for key, val in dict_k_s_y.items() if key != k]).reset_index(drop=True)

    beta_hat = get_linear_regression_model(train_set, s_y_train)
    s_y_hat = predict_linear_regression_value(beta_hat, test_set)
    
    mae = np.append( mae, get_mae(s_y,s_y_hat) )

In [17]:
print("The min MAE is {:.2f}, the max MAE is {:.2f}, and the mean MAE is {:.2f}".format(mae.min(),mae.max(),mae.mean()))

The min MAE is 4.30, the max MAE is 11.03, and the mean MAE is 8.18


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your class label (the dependent variable) series `s_y`

In [18]:
x,y = load_iris(return_X_y=True)
df_X = pd.DataFrame(x)
s_y = pd.DataFrame(y)

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Note, you can reuse the functions from Section 1. 

In [19]:
(dict_k_df_X, dict_k_s_y) = partition_data(df_X, s_y, 5)

In [20]:
for k in dict_k_df_X.keys():
  print("Fold {} length of dataframe is {} and length of series is {}".format(k, len(dict_k_df_X[k]), len(dict_k_s_y[k])))

Fold 1 length of dataframe is 25 and length of series is 25
Fold 2 length of dataframe is 36 and length of series is 36
Fold 3 length of dataframe is 27 and length of series is 27
Fold 4 length of dataframe is 31 and length of series is 31
Fold 5 length of dataframe is 31 and length of series is 31


## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [21]:
def get_acc( s_1, s_2 ):
    return sum(s_1.eq(s_2))/len(s_1)

In [22]:
get_acc(s_y.squeeze(),np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each of the 5 executions of the inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [23]:
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])

# Outer loop
outer_acc = np.array([])
for k in dict_k_df_X.keys():

    test_set = dict_k_df_X[k]
    s_y_test = dict_k_s_y[k]
    inner_acc = np.array([])
    
    for pos_min_impurity in possible_min_impurity_decrease:
        print("Testing {:.2f} min impurity decrease".format(pos_min_impurity))

        inner_folds = {i: dict_k_df_X[i] for i in dict_k_df_X.keys() if i != k}
        s_y_inner = {i: dict_k_s_y[i] for i in dict_k_s_y.keys() if i != k}
        fold_acc = np.array([])

        for i in inner_folds.keys():
            train_set = pd.concat([val for key, val in inner_folds.items() if key != i]).reset_index(drop=True)
            s_y_train = pd.concat([val for key, val in s_y_inner.items() if key != i]).reset_index(drop=True)

            clf = tree.DecisionTreeClassifier(criterion="gini", min_impurity_decrease=pos_min_impurity)
            clf.fit(train_set, s_y_train)
            fold_acc = np.append(fold_acc, get_acc(s_y_inner[i].squeeze(), clf.predict(inner_folds[i])))
        
        inner_acc = np.append(inner_acc, np.average(fold_acc))
        print("    Average accuracy over 4 folds is {:.2f}".format(inner_acc[-1]))

    best_min_impurity_decrease = possible_min_impurity_decrease[np.argmax(inner_acc)]
    print("\nBest min impurity decrease is {}\n".format(best_min_impurity_decrease))

    # Use best min impurity decrease to train model
    train_set = pd.concat([val for key, val in inner_folds.items() if key != k]).reset_index(drop=True)
    s_y_train = pd.concat([val for key, val in s_y_inner.items() if key != k]).reset_index(drop=True)

    clf = tree.DecisionTreeClassifier(criterion="gini", min_impurity_decrease=best_min_impurity_decrease)
    clf.fit(train_set, s_y_train)
    this_acc = get_acc(s_y_test.squeeze(), clf.predict(test_set))
    
    # outer accuracy calculation
    outer_acc = np.append(outer_acc,this_acc) # make sure and calculate this_acc in your loop

Testing 0.10 min impurity decrease
    Average accuracy over 4 folds is 0.93
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.93
Testing 0.30 min impurity decrease
    Average accuracy over 4 folds is 0.65
Testing 0.40 min impurity decrease
    Average accuracy over 4 folds is 0.33

Best min impurity decrease is 0.1

Testing 0.10 min impurity decrease
    Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.78
Testing 0.30 min impurity decrease
    Average accuracy over 4 folds is 0.64
Testing 0.40 min impurity decrease
    Average accuracy over 4 folds is 0.30

Best min impurity decrease is 0.1

Testing 0.10 min impurity decrease
    Average accuracy over 4 folds is 0.94
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.87
Testing 0.30 min impurity decrease
    Average accuracy over 4 folds is 0.68
Testing 0.40 min impurity decrease
    Average accuracy over 4 folds is 0.32

Bes

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [24]:
print("Outer Fold Test Set Performance\nMin: {:.3f}, Max: {:.3f}, Mean: {:.3f}".format(np.amin(outer_acc), np.amax(outer_acc), np.average(outer_acc)))

Outer Fold Test Set Performance
Min: 0.889, Max: 1.000, Mean: 0.937
